# What Features Affect a Borrower's Loan Rate and Prosper Rating
## by Alf Maglalang

## Investigation Overview

> The goal of the investigation is to examine through visualizations which features have relationships with **Prosper Rating** and `BorrowerRate`. I am conflating **Prosper Rating** to mean the features `CreditGrade` and `ProsperRating (numeric)` which are mutually exclusive with `CreditGrade` before mid 2009 and `ProsperRating (numeric)` after mid-2009.

## Dataset Overview

> This loan data is from a lending company called **Prosper**. It was last updated 3/11/2014. It has 113937 records with 81 features, namely

> `ListingKey`, `ListingNumber`, `ListingCreationDate`, `CreditGrade`, `Term`, `LoanStatus`, `ClosedDate`, `BorrowerAPR`, `BorrowerRate`, `LenderYield`, `EstimatedEffectiveYield`, `EstimatedLoss`, `EstimatedReturn`, `ProsperRating (numeric)`, `ProsperRating (Alpha)`, `ProsperScore`, `ListingCategory (numeric)`, `BorrowerState`, `Occupation`, `EmploymentStatus`, `EmploymentStatusDuration`, `IsBorrowerHomeowner`, `CurrentlyInGroup`, `GroupKey`, `DateCreditPulled`, `CreditScoreRangeLower`, `CreditScoreRangeUpper`, `FirstRecordedCreditLine`, `CurrentCreditLines`, `OpenCreditLines`, `TotalCreditLinespast7years`, `OpenRevolvingAccounts`, `OpenRevolvingMonthlyPayment`, `InquiriesLast6Months`, `TotalInquiries`, `CurrentDelinquencies`, `AmountDelinquent`, `DelinquenciesLast7Years`, `PublicRecordsLast10Years`, `PublicRecordsLast12Months`, `RevolvingCreditBalance`, `BankcardUtilization`, `AvailableBankcardCredit`, `TotalTrades`, `TradesNeverDelinquent (percentage)`, `TradesOpenedLast6Months`, `DebtToIncomeRatio`, `IncomeRange`, `IncomeVerifiable`, `StatedMonthlyIncome`, `LoanKey`, `TotalProsperLoans`, `TotalProsperPaymentsBilled`, `OnTimeProsperPayments`, `ProsperPaymentsLessThanOneMonthLate`, `ProsperPaymentsOneMonthPlusLate`, `ProsperPrincipalBorrowed`, `ProsperPrincipalOutstanding`, `ScorexChangeAtTimeOfListing`, `LoanCurrentDaysDelinquent`, `LoanFirstDefaultedCycleNumber`, `LoanMonthsSinceOrigination`, `LoanNumber`, `LoanOriginalAmount`, `LoanOriginationDate`, `LoanOriginationQuarter`, `MemberKey`, `MonthlyLoanPayment`, `LP_CustomerPayments`, `LP_CustomerPrincipalPayments`, `LP_InterestandFees`, `LP_ServiceFees`, `LP_CollectionFees`, `LP_GrossPrincipalLoss`, `LP_NetPrincipalLoss`, `LP_NonPrincipalRecoverypayments`, `PercentFunded`, `Recommendations`, `InvestmentFromFriendsCount`, `InvestmentFromFriendsAmount`, `Investors`

> Please click on the following link to see the definitions of the data's features. [Prosper Data Dictionary to Explain Dataset's Variables](https://www.google.com/url?q=https://docs.google.com/spreadsheet/ccc?key%3D0AllIqIyvWZdadDd5NTlqZ1pBMHlsUjdrOTZHaVBuSlE%26usp%3Dsharing&sa=D&ust=1554486256024000)

> For my analysis, I will look at these features:
> `ListingKey`, `EmploymentStatusDuration`, `EmploymentStatus`, `LoanOriginalAmount`, `CreditGrade`, `ProsperRating (numeric)`, `ProsperScore`, `BorrowerRate`, `CreditScoreRangeUpper`, `CurrentDelinquencies`, `IsBorrowerHomeowner`, `CurrentCreditLines`, `AvailableBankcardCredit`

> I created 3 new features and renamed one..

> - `ProsperRating` -- renaming of `ProsperRating (numeric)` 
> - `ProsperScoreRange` -- a column containing a string of "incorrect" `ProsperScore`s of duplicated `ListingKey` IDs
> - `ProsperRatingGrade` -- a column combining `CreditGrade` and `ProsperRating`
> - `post2009` -- a column that is a form of boolean mask differentiating if a `ProsperRatingGrade` is originally from `CreditGrade` or from `ProsperRating`.

In [1]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

# suppress warnings from final output
import warnings
warnings.simplefilter("ignore")

In [10]:
# load in the dataset into a pandas dataframe
# pld will be short for prosperLoanData
pld = pd.read_csv('prosperLoanData.csv')

## Remove Duplicate ListingKey IDs (while preserving ProsperScore)

In [11]:
# dup_listing_idx contains the keys that have duplicates:
dup_idx = pld[pld.ListingKey.duplicated()]['ListingKey']
dup_listing_idx = set(dup_idx.tolist())

In [12]:
tpld = pld.copy()

In [14]:
# trim spaces to make sure it is not spaces that make a difference
tpld = tpld.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# create a dataframe with just ListingKey and ProsperScore
lk_ps_df = tpld[tpld.ListingKey.isin(dup_listing_idx)][['ListingKey', 'ProsperScore']]
lk_ps_df = lk_ps_df.sort_values(by=['ListingKey', 'ProsperScore'])

# create a new column ProsperScoreRange and set index to ListingKey
lk_ps_df['ProsperScoreRange'] = None
lk_ps_df = lk_ps_df.set_index(['ListingKey'])

# create a dictionary with key as ListingKey and the ProsperScore range as value
psr_dict = {}
for lk in dup_listing_idx:
    psr_dict[lk] = lk_ps_df.loc[lk]['ProsperScore'].tolist()

# place all the new range value from the dictionary into dataframe column lk_ps_df['ProsperScoreRange']
for k, v in psr_dict.items():
    vstr = [str(i) for i in v] # convert the list into strings
    yy = ",".join(vstr)
    lk_ps_df.loc[k, 'ProsperScoreRange'] = yy

# reset the index then drop the column ProsperScore from lk_ps_df
lk_ps_df.reset_index(inplace=True)
lk_ps_df = lk_ps_df.drop(['ProsperScore'], axis=1)

# now remove all the duplicates on ListingKey
lk_ps_df.drop_duplicates(subset=['ListingKey'], inplace=True, keep='first')

temp_cols = ['ListingKey', 'ListingNumber', 'ListingCreationDate', 'CreditGrade', 'Term', 'LoanStatus', 'ClosedDate', 'BorrowerAPR', 'BorrowerRate', 'LenderYield', 'EstimatedEffectiveYield', 'EstimatedLoss', 'EstimatedReturn', 'ProsperRating (numeric)', 'ProsperRating (Alpha)', 'ListingCategory (numeric)', 'BorrowerState', 'Occupation', 'EmploymentStatus', 'EmploymentStatusDuration', 'IsBorrowerHomeowner', 'CurrentlyInGroup', 'GroupKey', 'DateCreditPulled', 'CreditScoreRangeLower', 'CreditScoreRangeUpper', 'FirstRecordedCreditLine', 'CurrentCreditLines', 'OpenCreditLines', 'TotalCreditLinespast7years', 'OpenRevolvingAccounts', 'OpenRevolvingMonthlyPayment', 'InquiriesLast6Months', 'TotalInquiries', 'CurrentDelinquencies', 'AmountDelinquent', 'DelinquenciesLast7Years', 'PublicRecordsLast10Years', 'PublicRecordsLast12Months', 'RevolvingCreditBalance', 'BankcardUtilization', 'AvailableBankcardCredit', 'TotalTrades', 'TradesNeverDelinquent (percentage)', 'TradesOpenedLast6Months', 'DebtToIncomeRatio', 'IncomeRange', 'IncomeVerifiable', 'StatedMonthlyIncome', 'LoanKey', 'TotalProsperLoans', 'TotalProsperPaymentsBilled', 'OnTimeProsperPayments', 'ProsperPaymentsLessThanOneMonthLate', 'ProsperPaymentsOneMonthPlusLate', 'ProsperPrincipalBorrowed', 'ProsperPrincipalOutstanding', 'ScorexChangeAtTimeOfListing', 'LoanCurrentDaysDelinquent', 'LoanFirstDefaultedCycleNumber', 'LoanMonthsSinceOrigination', 'LoanNumber', 'LoanOriginalAmount', 'LoanOriginationDate', 'LoanOriginationQuarter', 'MemberKey', 'MonthlyLoanPayment', 'LP_CustomerPayments', 'LP_CustomerPrincipalPayments', 'LP_InterestandFees', 'LP_ServiceFees', 'LP_CollectionFees', 'LP_GrossPrincipalLoss', 'LP_NetPrincipalLoss', 'LP_NonPrincipalRecoverypayments', 'PercentFunded', 'Recommendations', 'InvestmentFromFriendsCount', 'InvestmentFromFriendsAmount', 'Investors']

# drop all the duplicates in tpld based on temp_cols subset (i.e., without ProsperScore)
tpld.drop_duplicates(subset=temp_cols, inplace=True, keep='first')

# merge the two df tpld and lk_ps_df on ListingKey
tpld = pd.merge(tpld, lk_ps_df, on=['ListingKey'], how='left')

## Extract the investigative features

In [20]:
select_features = ['ListingKey', 'EmploymentStatusDuration', 'EmploymentStatus', 'LoanOriginalAmount', 'CreditGrade', 'ProsperRating (numeric)', 'ProsperScore', 'ProsperScoreRange', 'BorrowerRate', 'CreditScoreRangeUpper', 'CurrentDelinquencies', 'IsBorrowerHomeowner', 'CurrentCreditLines', 'AvailableBankcardCredit']
df = tpld[select_features]
df = df.rename(columns={'ProsperRating (numeric)': 'ProsperRating', 'CreditScoreRangeUpper': 'CreditScore'})

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113066 entries, 0 to 113065
Data columns (total 14 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ListingKey                113066 non-null  object 
 1   EmploymentStatusDuration  105441 non-null  float64
 2   EmploymentStatus          110811 non-null  object 
 3   LoanOriginalAmount        113066 non-null  int64  
 4   CreditGrade               28953 non-null   object 
 5   ProsperRating             83982 non-null   float64
 6   ProsperScore              83982 non-null   float64
 7   ProsperScoreRange         827 non-null     object 
 8   BorrowerRate              113066 non-null  float64
 9   CreditScore               112475 non-null  float64
 10  CurrentDelinquencies      112369 non-null  float64
 11  IsBorrowerHomeowner       113066 non-null  bool   
 12  CurrentCreditLines        105462 non-null  float64
 13  AvailableBankcardCredit   105522 non-null  f

In [23]:
df.head()

,ListingKey,EmploymentStatusDuration,EmploymentStatus,LoanOriginalAmount,CreditGrade,ProsperRating,ProsperScore,ProsperScoreRange,BorrowerRate,CreditScore,CurrentDelinquencies,IsBorrowerHomeowner,CurrentCreditLines,AvailableBankcardCredit
0,1021339766868145413AB3B,2.0,Self-employed,9425,C,NaN,NaN,NaN,0.1580,659.0,2.0,True,5.0,1500.0
1,10273602499503308B223C1,44.0,Employed,10000,NaN,6.0,7.0,NaN,0.0920,699.0,0.0,False,14.0,10266.0
2,0EE9337825851032864889A,NaN,Not available,3001,HR,NaN,NaN,NaN,0.2750,499.0,1.0,False,NaN,NaN
3,0EF5356002482715299901A,113.0,Employed,10000,NaN,6.0,9.0,NaN,0.0974,819.0,4.0,True,5.0,30754.0
4,0F023589499656230C5E3E2,44.0,Employed,15000,NaN,3.0,4.0,NaN,0.2085,699.0,0.0,True,19.0,695.0


## Create a boolean column to differentiate pre-2009 and post-2009 ratings

### Please note that this is to facilitate plot drawing

In [33]:
# create a boolean mask IsCG IsPR
# create a column IsPR
wdf = df.copy()

In [36]:
# creating mask columns for ProsperRating and CreditGrade
df['IsPR'] = np.where(df.ProsperRating.notnull(), 1, None) # column IsPR
df['IsCG'] = np.where(df.CreditGrade.notnull(), 2, None) # column 

# create column post2009 with copy of IsPR first, then update to include IsCG
df['post2009'] = df['IsPR']
df.post2009.update(df['IsCG'])
no_rating = df.post2009.isnull().sum()

In [38]:
df[['IsPR', 'ProsperRating', 'IsCG', 'CreditGrade', 'post2009']].sample(1300).head(20)

,IsPR,ProsperRating,IsCG,CreditGrade,post2009
48823,1,6.0,None,NaN,1
57358,1,6.0,None,NaN,1
81356,1,4.0,None,NaN,1
8180,1,4.0,None,NaN,1
106742,1,5.0,None,NaN,1
110598,1,4.0,None,NaN,1
75688,1,5.0,None,NaN,1
35939,None,NaN,2,B,2
68999,1,5.0,None,NaN,1
42897,None,NaN,2,A,2


In [39]:
# drop all the rows that have nulls in post2009, i.e. no content in either CreditGrade nor ProsperRating
df = df.dropna(subset=['post2009'])

# replace all the 2 (the CreditGrade content) to 0
df.post2009.replace(2, 0, inplace=True)

df[['IsPR', 'ProsperRating', 'IsCG', 'CreditGrade', 'post2009']].sample(1300).head(20)

,IsPR,ProsperRating,IsCG,CreditGrade,post2009
22538,1,5.0,None,NaN,1
58889,1,4.0,None,NaN,1
6763,None,NaN,2,HR,0
56121,1,7.0,None,NaN,1
52886,1,6.0,None,NaN,1
96010,1,3.0,None,NaN,1
112182,1,3.0,None,NaN,1
104966,1,5.0,None,NaN,1
107924,1,3.0,None,NaN,1
22893,1,5.0,None,NaN,1


In [40]:
df.shape

(112935, 17)

In [43]:
# number of df records minus records with null in ProsperRatingGrade
tpld.shape[0]- df.ProsperRatingGrade.isnull().sum()[0]
# 113066-131

AttributeError: 'DataFrame' object has no attribute 'ProsperRatingGrade'

> Note that the above cells have been set as "Skip"-type slides. That means
that when the notebook is rendered as http slides, those cells won't show up.

## What is ProsperScore to ProsperRating

> The journey began with the discovery that there were duplicate records that only differed in the `ProsperScore` column. To correct these "errors", I set to determine and possibly use the relationship between `ProsperRating` and `ProsperScore` since `ProsperRating` is the most correlative to `ProsperScore`.
> The visual below illustrate that `ProsperRating` is a reasonable measure to determine "correct" `ProsperScore`.

## (Visualization 2)

> You should have at least three visualizations in your presentation,
but feel free to add more if you'd like!

## (Visualization 3)



> Once you're ready to finish your presentation, check your output by using
nbconvert to export the notebook and set up a server for the slides. From the
terminal or command line, use the following expression:
> > `jupyter nbconvert <file_name>.ipynb --to slides --post serve --template output_toggle`

> This should open a tab in your web browser where you can scroll through your
presentation. Sub-slides can be accessed by pressing 'down' when viewing its parent
slide. Make sure you remove all of the quote-formatted guide notes like this one
before you finish your presentation!